<a href="https://colab.research.google.com/github/JandreiSS/Analise-Fundamentalista/blob/master/DataFrame_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import requests
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
url = 'http://fundamentus.com.br/resultado.php'

In [4]:
r = requests.get(url, headers=header)

In [5]:
df = pd.read_html(r.text, thousands='.', decimal=',')[0]

In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

In [7]:
df = df[df['Liq.2meses'] > 1000000]

In [8]:
ranking = pd.DataFrame()
ranking['Position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [9]:
auxA = ranking.pivot_table(columns='EV/EBIT', values='Position')
auxB = ranking.pivot_table(columns='ROIC', values='Position')
t = pd.concat([auxA, auxB])

In [29]:
rank = t.dropna(axis=1).sum().astype(int).sort_values()

In [12]:
rank.to_csv('ranking.csv', index=True, sep=';')

In [13]:
df = df.loc[:,['Papel']]

In [14]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [15]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 3) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("\nNão foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [16]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values

In [17]:
requisicoes = requestUrls(urls, timeout=60)

In [18]:
lista_empresas = []

In [19]:
total_ticket = len(df)

In [20]:
for requisicao in tqdm(requisicoes, unit=' requisições', desc='Requisitando dados: ', disable=False, total=total_ticket, ncols=100):
    # codigo = requisicao.status_code
    # url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    # print (f'{codigo}: {url}')

Requisitando dados: 100%|███████████████████████████████| 212/212 [01:45<00:00,  2.02 requisições/s]


### Transformar os dados do HTML para um DataFrame

In [21]:
colunas = [
    'Papel',
    'Empresa',
    'Setor',
    'Subsetor',
    'Cotação',
    'Valor da Firma',
    'Valor de Mercado',
    'Número de Ações',
    'P/L',
    'Div. Yield',
    'EV/EBITDA',
    'EV/EBIT',
    'ROE',
    'ROIC'
]

In [22]:
tabela_base = pd.DataFrame(columns=colunas)

In [23]:
total_lista = len(lista_empresas)

In [24]:
for key in tqdm(range(total_lista), unit=' dados', desc='Tratando dados: ', disable=False, total=total_lista, ncols=100):
    dados_base = pd.read_html(lista_empresas[key])
    papel = dados_base[0][1][0]
    empresa = dados_base[0][1][2]
    setor = dados_base[0][1][3]
    subsetor = dados_base[0][1][4]
    cotacao = dados_base[0][3][0]
    valor_da_firma = dados_base[1][1][1]
    valor_de_mercado = dados_base[1][1][0]
    n_acoes = dados_base[1][3][1]
    p_l = dados_base[2][3][1]
    div_yield = dados_base[2][3][8]
    ev_ebitda = dados_base[2][3][9]
    ev_ebit = dados_base[2][3][10]
    roe = dados_base[2][5][8]
    roic = dados_base[2][5][7]
    to_append = [papel, empresa, setor, subsetor, cotacao, valor_da_firma, valor_de_mercado, n_acoes, p_l, div_yield, ev_ebitda, ev_ebit, roe, roic]
    tabela_base_length = len(tabela_base)
    tabela_base.loc[tabela_base_length] = to_append


Tratando dados: 100%|█████████████████████████████████████████| 212/212 [00:11<00:00, 19.09 dados/s]


In [25]:
for coluna in ['ROE', 'Div. Yield', 'ROIC']:
    tabela_base[coluna] = tabela_base[coluna].str.rstrip('%')

In [27]:
tabela_base.to_csv('tabela_dados.csv', index=False, sep=';')